In [2]:
import pandas as pd
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.ext.automap import automap_base
import numpy as np
from pprint import pprint



from flask import (
    Flask,
    render_template,
    jsonify,
    request,
    redirect)



dataset1 = pd.read_excel("BronxPropertySalesDatasets/sales_bronx_03.xls")

In [33]:
data_03 = pd.read_excel ('BronxPropertySalesDatasets/sales_bronx_03.xls')
data_04= pd.read_excel ('BronxPropertySalesDatasets/sales_bronx_04.xls')
data_05= pd.read_excel ('BronxPropertySalesDatasets/sales_bronx_05.xls')
data_06= pd.read_excel ('BronxPropertySalesDatasets/sales_bronx_06.xls')
data_07= pd.read_excel ('BronxPropertySalesDatasets/sales_2007_bronx.xls')
data_08= pd.read_excel ('BronxPropertySalesDatasets/sales_2008_bronx.xls')
data_09= pd.read_excel ('BronxPropertySalesDatasets/2009_bronx.xls')
data_10= pd.read_excel ('BronxPropertySalesDatasets/2010_bronx.xls')
data_11= pd.read_excel ('BronxPropertySalesDatasets/2011_bronx.xls')
data_12= pd.read_excel ('BronxPropertySalesDatasets/2012_bronx.xls')
data_13= pd.read_excel ('BronxPropertySalesDatasets/2013_bronx.xls')
data_14= pd.read_excel ('BronxPropertySalesDatasets/2014_bronx.xls')
data_15= pd.read_excel ('BronxPropertySalesDatasets/2015_bronx.xls')
data_16= pd.read_excel ('BronxPropertySalesDatasets/2016_bronx.xls')
data_17= pd.read_excel ('BronxPropertySalesDatasets/2017_bronx.xls')
data_18= pd.read_excel ('BronxPropertySalesDatasets/2018_bronx.xlsx')

In [31]:
new_header = data_03.iloc[2].values
data_03 = data_03 [3:]
data_03.columns = new_header


#new_header = df.iloc[0] #grab the first row for the header
#df = df[1:] #take the data less the header row
#df.columns = new_header #set the header row as the df header


In [61]:
def prepare_data ( df ):
    
    new_header = df.iloc[2].values
    df = df [3:]
    df.columns = new_header
    
    df.loc[ df ['GROSS SQUARE FEET'] != 0]
    df.loc[ df ['SALE PRICE'] > 25000]
    df.loc[ df ['SALE PRICE'] !=0]
    # ----------------------------------------------------------------------------- #
    
    sqft_bins = np.arange (0,500000, 300)
    sqft_bins = list(sqft_bins )
    sq_ft_cat = pd.cut( df ['GROSS SQUARE FEET'], sqft_bins )
    # ----------------------------------------------------------------------------- #
    df ['SQUARE FEET CATEGORY'] = sq_ft_cat 
    df ['YEAR'] = df ['SALE DATE'][:4] 
    
    return df

df_03 = prepare_data (data_03)
df_04 = prepare_data (data_04)



/Users/diegoleoro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Users/diegoleoro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [102]:
df_03 = df_03[df_03["SALE PRICE"]!=0]

In [106]:
df_03.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,SQUARE FEET CATEGORY,YEAR
4,2,BATHGATE,01 ONE FAMILY HOMES,1,2917,15,,A1,1848 WASHINGTON AVENUE,,...,1,2943,2128,1920,1,A1,130000,2003-05-27 00:00:00,"(2100, 2400]",2003-05-27 00:00:00
5,2,BATHGATE,01 ONE FAMILY HOMES,1,3028,25,,A5,412 EAST 179 STREET,,...,1,1842,2048,1901,1,A1,204000,2003-04-07 00:00:00,"(1800, 2100]",2003-04-07 00:00:00
6,2,BATHGATE,01 ONE FAMILY HOMES,1,3030,55,,A5,410 EAST 182 STREET,,...,1,1330,1460,1899,1,A1,235000,2003-07-24 00:00:00,"(1200, 1500]",2003-07-24 00:00:00
7,2,BATHGATE,01 ONE FAMILY HOMES,1,3035,2,,S1,441 EAST 178 STREET,,...,2,1287,2378,1899,1,S1,125500,2003-05-02 00:00:00,"(2100, 2400]",NaN
8,2,BATHGATE,01 ONE FAMILY HOMES,1,3035,52,,A1,461 EAST 178 STREET,,...,1,1782,1548,1899,1,A1,215000,2003-05-14 00:00:00,"(1500, 1800]",NaN


In [128]:
df_03.corr()

""


In [94]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split 

In [104]:
X = df_03[['GROSS SQUARE FEET']]
y = df_03['SALE PRICE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
regressor = LinearRegression().fit(X, y)
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)


In [105]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df

,Actual,Predicted
5406,270000,348037.265263
4605,205822,341325.697689
5833,73000,327786.845860
6949,146000,327786.845860
4722,330000,338965.077370
...,...,...
10473,205000,348037.265263
8113,365000,344970.773182
5413,250000,343061.447924
9509,370800,345144.348205


In [111]:
from sklearn.datasets import make_regression

X = df_03[['GROSS SQUARE FEET']]
y = df_03['SALE PRICE']

model = LinearRegression()

model.fit(X, y)

score = model.score(X, y)
print(f"R2 Score: {score}")



R2 Score: 0.05947656901589449


In [112]:
from sklearn import preprocessing

In [114]:
df03 = df_03[["NEIGHBORHOOD", "BUILDING CLASS CATEGORY"]]

In [132]:
le = preprocessing.LabelEncoder()

X_03 = df03.apply(le.fit_transform)
X_03 ["SALE PRICE"] = df_03 ["SALE PRICE"]
X_03 ["SQUARE FEET"] = df_03 ["GROSS SQUARE FEET"]

X = X_03 [['SQUARE FEET']]
y = df_03['SALE PRICE']


model = LinearRegression()

model.fit(X, y)

score = model.score(X, y)
print(f"R2 Score: {score}")



R2 Score: 0.05947656901589449


In [126]:
X_03

,NEIGHBORHOOD,BUILDING CLASS CATEGORY,SALE PRICE,SQUARE FEET
4,0,0,130000,2128
5,0,0,204000,2048
6,0,0,235000,1460
7,0,0,125500,2378
8,0,0,215000,1548
...,...,...,...,...
10661,35,9,102902,0
10662,35,9,74696,0
10663,35,9,35490,0
10664,35,15,750000,2900


In [ ]:
values_for_df = [ ]
def price_avg_sqft_cat ( df ):
	sqft_cat_unique_values =df [“SQUARE FEET CATEGORY”].uniqie( )
	for category in sqft_cat_unique_values:
		x = df [df [“SQUARE FEET CATEGORY”] == category ]
		category_price_avg = x [ “SALE PRICE” ].mean( )
		year = df.iloc[“0”][“ YEAR”]
		values = [year,  category, category_price_avg]
		values_for_df.append (values)



df = pd.DataFrame(data, columns = [Year, 'Square Feet Category', ‘Average Price Sale’])


In [ ]:
#df ['SALE PRICE']  =  df ['SALE PRICE'].str.replace('$','')
#df ['SALE PRICE']  =  df ['SALE PRICE'].str.replace(',', '')
#df ['SALE PRICE']  =  df ['SALE PRICE'].astype(int) 
    
#df ['GROSS SQUARE FEET']  =  df ['GROSS SQUARE FEET'].str.replace(',', '')
#df ['GROSS SQUARE FEET']  =  df ['GROSS SQUARE FEET'].astype(int) 